In [1]:
!pip install torch transformers datasets

In [2]:
!pip install transformers datasets torch


In [3]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import torch


In [4]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import torch

# Load the dataset from a local file (use the provided dataset or a custom one)
dataset = load_dataset("json", data_files="/kaggle/input/qadataset/qa_dataset.json")

# Use a multilingual model like XLM-RoBERTa (supports both English and Bengali)
model_name = "xlm-roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)


Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
def preprocess_function(examples):
    # Use the correct column names based on the dataset inspection
    questions = examples["question"]  # Column name for questions
    contexts = examples["answer"]     # Column name for contexts (answers)
    
    # Tokenize the question and context together
    inputs = tokenizer(questions, contexts, truncation=True, padding=True, max_length=512)
    
    # Add the start and end positions for the answer span
    start_positions = []
    end_positions = []
    for i, context in enumerate(contexts):
        answer = examples["answer"][i]  # Replace with the correct answer column
        start_pos = context.find(answer)
        end_pos = start_pos + len(answer) - 1
        start_positions.append(start_pos)
        end_positions.append(end_pos)
    
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    
    return inputs

# Apply preprocessing
tokenized_datasets = dataset.map(preprocess_function, batched=True)


In [6]:
# Split the dataset into train and validation sets
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(tokenized_datasets['train'], test_size=0.2)

# Convert the split data into Hugging Face dataset format
from datasets import Dataset
train_dataset = Dataset.from_dict(train_data)
test_dataset = Dataset.from_dict(test_data)

# Update the tokenized_datasets to include the new 'test' split
tokenized_datasets['test'] = test_dataset
training_args = TrainingArguments(
    output_dir="./qa_model",         # Directory to save the model
    eval_strategy="epoch",           # Evaluate at the end of each epoch
    save_strategy="epoch",           # Save the model at the end of each epoch
    learning_rate=2e-5,              # Learning rate
    per_device_train_batch_size=8,   # Batch size for training
    per_device_eval_batch_size=8,    # Batch size for evaluation
    num_train_epochs=3,              # Number of epochs
    weight_decay=0.01,               # Weight decay to prevent overfitting
    logging_dir='./logs',            # Directory for logs
    logging_steps=10,                # Log every 10 steps
    load_best_model_at_end=True,     # Load the best model after training
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=tokenized_datasets['test'],  # Now use the 'test' split
    tokenizer=tokenizer,
)

# Fine-tune the model
trainer.train()
# Evaluate the model
trainer.evaluate()

# Save the fine-tuned model
trainer.save_model("./qa_model")




/tmp/ipykernel_170/3787968467.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


ResourceExhaustedError: logs/events.out.tfevents.1733461342.9d55b4f4cd37.170.0; No space left on device

In [ ]:
# Load the fine-tuned model
model = AutoModelForQuestionAnswering.from_pretrained("./qa_model")
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Test the model on a sample question-answer pair
question = "What is the capital of Bangladesh?"
context = "Bangladesh is a country located in South Asia. Its capital is Dhaka."

inputs = tokenizer(question, context, return_tensors="pt")
outputs = model(**inputs)

# Extract the start and end positions for the answer span
start_index = torch.argmax(outputs.start_logits)
end_index = torch.argmax(outputs.end_logits)

# Convert token indices back to text
answer_tokens = inputs.input_ids[0][start_index:end_index + 1]
answer = tokenizer.decode(answer_tokens)

print("Answer:", answer)
